In [1]:
from memory_reveal_cards_game import MemoryGame
from minimum_vertex_cover_vertices import MinimumVertexCoverVertices
from logChoice import LogChoice
from bigChoice import BigChoice
from fixedParameters import FixedParameters
from transformerParameters import TransformerParametersSequential
from pso import pso_solve
import numpy as np
import random
from utils import smallest_dominating_set, greedy_dominating_set, generate_random_graph

In [2]:
choice = LogChoice()
choiceParameters = FixedParameters()
problem = MemoryGame(choice, choiceParameters, 0.01, 20, 50)

In [3]:
def fitness(p, cards):
    total = 0
    count = 10
    ol = 0

    choiceParameters.set(p[0][0], p[0][1])
    
    for i in range(count):
        problem.prob_forget = random.random() * 0.03
        v = problem.run(cards)

        if v == problem.limit:
            ol += 1
        else:
            total += v

    if count > ol:
        return ol + 2 * (total / (count - ol)) / problem.limit
    else:
        return ol + 2

In [4]:
def get_sequences_from_start(list, amount):
    pos = set()
    n = len(list)

    while len(pos) < amount:
        pos.add(random.randint(0, n - 1))

    return [list[:poss + 1] for poss in pos]

In [9]:
X, Y = [], []

count = 10

for _ in range(count):
    init_positions = lambda num_particles: [np.random.random((1, 2)) * 2 - 1 for i in range(num_particles)]
    init_velocities = lambda num_particles: [np.zeros((1, 2)) for i in range(num_particles)]   

    cards = problem.random_cards()

    p, score, positions, velocities = pso_solve(10, 10
                                                , init_positions
                                                , init_velocities
                                                , lambda p: fitness(p, cards)
                                                , lambda p: choice.apply_constraints(p[0][0], p[0][1])
                                                , lambda score: score < 0.8
                                                , False
                                                )
    
    choiceParameters.set(p[0][0], p[0][1])
    
    for i in range(count):
        problem.prob_forget = random.random() * 0.03
        problem.run(cards)

        X.extend(get_sequences_from_start(problem.embeddings, 5) + [problem.embeddings])
        Y.extend([p[0]] * 6)

d:\SVD\tesis\logChoice.py:11: RuntimeWarning: overflow encountered in double_scalars
  exp = x ** (1 / b)


In [5]:
from pickle import load, dump

with open('xy', 'rb') as f:
    X, Y = load(f)

In [6]:
print(len(X), len(Y))

600 600


In [7]:
transformerParameters = TransformerParametersSequential()
preds, error = transformerParameters.train(X, Y)

print(error)

Epoch 1/10, Loss: 1.3536
Epoch 2/10, Loss: 0.3898
Epoch 3/10, Loss: 0.3615
Epoch 4/10, Loss: 0.3591
Epoch 5/10, Loss: 0.3504
Epoch 6/10, Loss: 0.3631
Epoch 7/10, Loss: 0.3645
Epoch 8/10, Loss: 0.3521
Epoch 9/10, Loss: 0.3559
Epoch 10/10, Loss: 0.3600
MSE: 0.4251
0.42510918


In [7]:
with open('tp', 'rb') as f:
    transformerParameters = load(f)

In [14]:
def test(choice):
    problem = MemoryGame(choice, transformerParameters, 0.03, 36, 1000)
    total = 0
    count = 100
    ol = 0
    
    for i in range(count):
        cards = problem.random_cards()
        v = problem.run(cards)

        if v == problem.limit:
            ol += 1
        else:
            total += v

    if count > ol:
        print(total / (count - ol), ol)
    else:
        print(ol)

In [15]:
test(LogChoice())

91.35 0


In [ ]:
test(BigChoice())